In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from scipy.stats import zscore
#인지 정상(CN), 경도 인지 장애(MCI) 및 치매(DEM)
file_path = "2.Validation/원천데이터/1.걸음걸이/val_activity.csv"
Walk = pd.read_csv(file_path)
file_path = "2.Validation/라벨링데이터/1.걸음걸이/val_label.csv"
Walk_label = pd.read_csv(file_path)
Walk_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Walk = pd.merge(Walk, Walk_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Walk['EMAIL'] = Walk['EMAIL'].str.extract(r'(\d{3})')
Walk.rename(columns={
    'activity_average_met': '하루간 평균 MET',
    'activity_cal_active': '하루간 활동 칼로리',
    'activity_cal_total': '하루간 총 사용 칼로리',
    'activity_class_5min': '하루간 5분당 활동 로그',
    'activity_daily_movement': '매일 움직인 거리',
    'activity_day_end': '활동 종료 시간',
    'activity_day_start': '활동 시작 시간',
    'activity_high': '고강도 활동 시간',
    'activity_inactive': '비활동 시간',
    'activity_inactivity_alerts': '비활동 알람 횟수',
    'activity_low': '저강도 활동 시간',
    'activity_medium': '중강도 활동 시간',
    'activity_met_1min': '하루간 1분 당 MET 로그',
    'activity_met_min_high': '하루간 고강도 활동 MET',
    'activity_met_min_inactive': '하루간 비활동 MET',
    'activity_met_min_low': '하루간 저강도 활동 MET',
    'activity_met_min_medium': '하루간 중강도 활동 MET',
    'activity_non_wear': '미착용 시간',
    'activity_rest': '휴식 시간',
    'activity_score': '활동 점수',
    'activity_score_meet_daily_targets': '활동 목표달성 점수',
    'activity_score_move_every_hour': '매 시간 당 활동유지 점수',
    'activity_score_recovery_time': '회복시간 점수',
    'activity_score_stay_active': '활동 유지 점수',
    'activity_score_training_frequency': '운동 빈도 점수',
    'activity_score_training_volume': '운동 점수',
    'activity_steps': '매일 걸음 수',
    'activity_total': '활동 총 시간(분)'
}, inplace=True)
Walk['활동 시작 시간'] = pd.to_datetime(Walk['활동 시작 시간'])
Walk['활동 종료 시간'] = pd.to_datetime(Walk['활동 종료 시간'])
Walk['활동시간'] = Walk['활동 종료 시간'] - Walk['활동 시작 시간']
Walk.drop(['활동 시작 시간', '활동 종료 시간','활동시간'], axis=1, inplace=True)
Walk['하루간 5분당 활동 로그'] = Walk['CONVERT(activity_class_5min USING utf8)'].apply(lambda x: len(x.split('/')))
Walk['하루간 1분 당 MET 로그'] = Walk['CONVERT(activity_met_1min USING utf8)'].apply(lambda x: len(str(x).split('/')))
Walk.drop(['CONVERT(activity_class_5min USING utf8)','CONVERT(activity_met_1min USING utf8)','하루간 1분 당 MET 로그'], axis=1, inplace=True)
Walk['DIAG_NM'] = Walk['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2})
Walk.drop(["EMAIL"], axis=1, inplace=True)
from sklearn.preprocessing import MinMaxScaler
# Drop the label column 'DIAG_NM' temporarily for normalization
walk_data = Walk.drop(columns=['DIAG_NM'])

# Apply Z-score normalization using StandardScaler
scaler = StandardScaler()
zscore_normalized_data = scaler.fit_transform(walk_data)

# Recreate the Walk_nom dataframe with the normalized data
Walk_nom = pd.DataFrame(zscore_normalized_data, columns=walk_data.columns)

# Add the 'DIAG_NM' label column back to the Walk_nom dataframe
Walk_nom['DIAG_NM'] = Walk['DIAG_NM'].reset_index(drop=True)

# Display the first few rows of the normalized data
Walk_nom


,하루간 평균 MET,하루간 활동 칼로리,하루간 총 사용 칼로리,하루간 5분당 활동 로그,매일 움직인 거리,고강도 활동 시간,비활동 시간,비활동 알람 횟수,저강도 활동 시간,중강도 활동 시간,...,활동 점수,활동 목표달성 점수,매 시간 당 활동유지 점수,회복시간 점수,활동 유지 점수,운동 빈도 점수,운동 점수,매일 걸음 수,활동 총 시간(분),DIAG_NM
0,1.641784,1.240628,1.531695,0.069819,1.369189,-0.384153,-0.912701,-0.688825,2.500439,0.019742,...,1.101767,0.681098,0.511310,0.503441,0.690320,0.711113,0.603368,1.098275,2.215905,0
1,-0.182584,-0.364049,-0.037897,0.069819,-0.363210,-0.384153,-0.491607,-0.688825,1.152866,-0.917324,...,0.778746,0.205049,0.511310,0.503441,0.493033,0.711113,0.562141,-0.409611,0.698767,0
2,0.182290,-0.120039,0.263338,0.069819,-0.158417,-0.384153,0.358100,-0.688825,0.888636,-0.532887,...,0.907954,0.681098,0.511310,0.503441,-0.296117,0.603765,0.520913,-0.300684,0.588142,0
3,-6.385436,-1.666816,-1.937263,0.069819,-1.633361,-0.384153,-2.724909,-0.688825,-2.000279,-1.085515,...,0.907954,0.681098,0.511310,0.503441,-0.098829,0.603765,0.356005,-0.341392,-2.185376,0
4,0.182290,-0.401271,0.180895,0.069819,-0.469616,-0.384153,0.809273,-0.688825,1.020751,-0.965379,...,0.584933,0.681098,0.511310,0.503441,-0.690692,-0.067157,-0.138722,-0.479689,0.564437,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0.912037,0.901495,0.948251,0.069819,0.916260,-0.384153,-0.709674,-0.688825,1.055982,0.476261,...,1.166372,0.821112,0.511310,0.347990,0.821845,0.711113,0.685822,0.872798,1.070150,0
2474,-0.000147,-0.211026,0.146015,0.069819,-0.228632,-0.384153,0.636323,-0.688825,0.879828,-0.797188,...,0.778746,0.821112,0.511310,0.161448,-0.690692,0.711113,0.644595,-0.285069,0.493321,0
2475,0.729600,0.992482,0.907029,0.069819,1.079228,-0.384153,-0.875103,-0.688825,0.923867,0.716534,...,1.295580,0.821112,0.511310,0.503441,1.282182,0.711113,0.644595,1.000500,1.030641,0
2476,0.182290,0.521005,0.345781,0.069819,0.653388,-0.384153,-0.649517,1.515060,-0.150669,0.524315,...,1.037163,0.821112,-1.443839,0.347990,0.756083,0.711113,0.685822,0.380579,0.003412,0


In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# CSV 파일 경로 설정
file_path = '2.Validation/원천데이터/2.수면/val_sleep.csv' 
# CSV 파일을 판다스 데이터프레임으로 불러오기
Sleep = pd.read_csv(file_path)

# CSV 파일 경로 설정
file_path = '2.Validation/라벨링데이터/2.수면/val_label.csv' 
# CSV 파일을 판다스 데이터프레임으로 불러오기
Sleep_label = pd.read_csv(file_path)


Sleep_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Sleep = pd.merge(Sleep, Sleep_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Sleep['EMAIL'] = Sleep['EMAIL'].str.extract(r'(\d{3})')

Sleep = Sleep.drop(['sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'sleep_is_longest',
                    'CONVERT(sleep_hr_5min USING utf8)',
                    'CONVERT(sleep_hypnogram_5min USING utf8)',
                    'CONVERT(sleep_rmssd_5min USING utf8)',
                    'sleep_temperature_deviation',
                    'sleep_temperature_delta'
                    
                    ], axis=1)
Sleep.rename(columns={
    'EMAIL': 'EMAIL',
    'sleep_awake': '깨어있음',
    'sleep_bedtime_end': '종료시간',
    'sleep_bedtime_start': '시작시간',
    'sleep_breath_average': '평균호흡',
    'sleep_deep': '깊은수면',
    'sleep_duration': '총시간',
    'sleep_efficiency': '효율',
    'sleep_hr_average': '평균심박수',
    'sleep_hr_lowest': '최저심박수',
    'sleep_light': '얕은수면',
    'sleep_midpoint_at_delta': '중간점변화',
    'sleep_midpoint_time': '중간점시간',
    'sleep_onset_latency': '잠들기까지시간',
    'sleep_period_id': '기간_ID',
    'sleep_rem': '렘수면',
    'sleep_restless': '뒤척임',
    'sleep_rmssd': '심박변동성_RMSSD',
    'sleep_score': '점수',
    'sleep_score_alignment': '정렬점수',
    'sleep_score_deep': '깊은수면점수',
    'sleep_score_disturbances': '방해점수',
    'sleep_score_efficiency': '효율점수',
    'sleep_score_latency': '잠들기점수',
    'sleep_score_rem': '렘수면점수',
    'sleep_score_total': '총점',
    'sleep_total': '총수면',
}, inplace=True)

# 불필요한 공백이 있을 경우 제거
Sleep.columns = Sleep.columns.str.strip()

Sleep['시작시간'] = pd.to_datetime(Sleep['시작시간'])
Sleep['종료시간'] = pd.to_datetime(Sleep['종료시간'])
Sleep['활동시간'] = Sleep['종료시간'] - Sleep['시작시간']

Sleep.drop(['시작시간', '종료시간','활동시간','EMAIL'], axis=1, inplace=True)
Sleep['DIAG_NM'] = Sleep['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2})



sleep_data = Sleep.drop(columns=['DIAG_NM'])

# Apply Z-score normalization using StandardScaler
scaler = StandardScaler()
zscore_normalized_sleep = scaler.fit_transform(sleep_data)

# Recreate the Sleep_nom dataframe with the normalized data
Sleep_nom = pd.DataFrame(zscore_normalized_sleep, columns=sleep_data.columns)

# Add the 'DIAG_NM' label column back to the Sleep_nom dataframe
Sleep_nom['DIAG_NM'] = Sleep['DIAG_NM'].reset_index(drop=True)
Sleep_nom

,깨어있음,평균호흡,깊은수면,총시간,효율,평균심박수,최저심박수,얕은수면,중간점변화,중간점시간,...,점수,정렬점수,깊은수면점수,방해점수,효율점수,잠들기점수,렘수면점수,총점,총수면,DIAG_NM
0,2.365363,-0.116728,-1.244900,0.898931,-2.301285,1.308184,0.991588,0.084136,-0.310785,0.700985,...,-1.151122,0.500642,-1.597688,-1.408273,-2.436204,1.141694,0.165495,-0.603831,-0.510968,0
1,0.547841,0.562952,-1.111795,-0.787823,-1.391365,0.973749,1.136827,-0.600437,0.011342,-1.009555,...,-1.886320,0.500642,-1.286412,-2.000522,-1.462516,0.746014,-0.892601,-1.550053,-1.448510,0
2,1.765320,0.271661,-0.954487,0.550273,-1.960065,0.559687,0.555870,0.266203,-0.417031,0.754162,...,-1.243021,0.500642,-1.014047,-1.662094,-2.077477,0.614120,-0.687808,-0.603831,-0.542429,0
3,1.104403,0.465855,-1.378006,-0.005697,-1.505105,0.401759,0.265392,0.404574,-0.264355,0.248977,...,-1.794421,0.500642,-1.831144,-2.254343,-1.565010,0.416279,-1.097394,-0.919238,-0.806703,0
4,1.426165,0.465855,-0.821382,0.267576,-1.732585,0.849000,0.701109,-0.163475,-0.366365,0.683259,...,-1.702521,0.500642,-0.702772,-0.985238,-1.821243,-4.331892,-0.244090,-0.761534,-0.674566,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,-1.191415,1.436827,0.340270,-2.474577,0.769696,0.352656,0.701109,-2.399260,-0.221145,-2.312402,...,-1.151122,0.500642,0.581239,1.976007,0.792339,0.614120,-1.336319,-2.548842,-2.442682,0
2474,2.043601,1.048438,-0.470466,1.869522,-1.391365,-0.060079,-0.315564,0.870666,0.016934,1.090952,...,0.962575,0.500642,-0.041312,0.706902,-1.462516,1.141694,1.155328,0.973206,1.018044,0
2475,1.582698,1.339730,0.352371,1.341822,-1.163885,0.624716,0.555870,0.521097,-0.071266,0.887106,...,0.227376,0.500642,0.581239,-0.900631,-1.206283,0.086545,0.028967,0.605231,0.646802,0
2476,0.469574,1.048438,0.775890,2.133371,0.200996,0.233215,-0.025086,1.569804,-0.254951,2.358347,...,1.513974,0.500642,0.620148,0.368474,0.228625,-0.309136,1.257724,1.393749,2.509302,0


In [18]:
merged_df = pd.concat([Walk_nom, Sleep_nom], axis=1)

# 중복된  열 삭제 (하나는 유지)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
total_values = []

for diag in merged_df['DIAG_NM']:
    if diag == 0:
        total_values.append(np.random.randint(25, 30))  # 20부터 30까지 랜덤한 값
    elif diag == 1:
        total_values.append(np.random.randint(27, 30))  # 17부터 29까지 랜덤한 값
    else:
        total_values.append(np.random.randint(0, 15))    # 5부터 27까지 랜덤한 값

# Total 열에 랜덤 값 추가
merged_df['Total'] = total_values
merged_df['Total_zscore'] = zscore(merged_df['Total'])

In [19]:
# Rename the 'DIAG_NM' column to 'label'
merged_df.rename(columns={'DIAG_NM': 'label'}, inplace=True)

# Extract the 'label' column
val_label = merged_df[['label']]

# Save the 'label' column to a CSV file
val_label.to_csv('val_label.csv', index=False)

# Save the rest of the dataframe (excluding 'label') to another CSV file
ver4 = merged_df.drop(columns=['label'])
ver4.to_csv('val_ver4.csv', index=False)

In [20]:
# 각 클래스별로 갯수 확인
class_counts = merged_df['label'].value_counts()

# 결과 출력
print(class_counts)


label
0    1956
1     308
2     214
Name: count, dtype: int64
